In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

[CRISP_DM](CRISP_DM.png)

## 4.1 Group-by Apply on large (relational) data set

- Attention: all writen functions assume a data frame where the date is sorted!

In [2]:
pd_JH_data = pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date', ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


## 4.2 Test data

In [3]:
test_data=pd_JH_data[((pd_JH_data['country']== 'US') | (pd_JH_data['country']== 'Germany')) & (pd_JH_data['date']>'2020-03-20')]

In [4]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15791,2020-03-21,no,US,25825.0
16002,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33761.0
16267,2020-03-23,no,Germany,29056.0
16307,2020-03-23,no,US,43850.0
16535,2020-03-24,no,Germany,32986.0
16575,2020-03-24,no,US,54112.0
16800,2020-03-25,no,Germany,37323.0
16841,2020-03-25,no,US,66055.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-07-15,no,200890.0
US,2020-07-15,no,3497847.0


### 4.2.1 Doubling time via regression_ calculation

In [6]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-07-15   200890.0
      US      2020-07-15  3497847.0

In [8]:
#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

### 4.2.3 Rolling regression

#### Thoery for understanding merge concept in python

- Inner Merge / Inner join – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right dataframes.
- Left Merge / Left outer join – (aka left merge or left join) Keep every row in the left dataframe. Where there are missing values of the “on” variable in the right dataframe, add empty / NaN values in the result.
- Right Merge / Right outer join – (aka right merge or right join) Keep every row in the right dataframe. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.
- Outer Merge / Full outer join – A full outer join returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with NaNs elsewhere.</font>

In [2]:
#Defining function for calculation of rolling regression
def rolling_reg(df_input,col='confirmed'):
    '''input has to be a data frame'''
    '''returna is single series (mandatory for group by apply) '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
    min_periods=days_back).apply(get_doubling_time_via_regression)
    return result

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg, 'confirmed')

state  country       
no     Germany  15736            NaN
                16002            NaN
                16267       7.417994
                16535       7.142035
                16800       8.012983
                17066       6.954407
                17334       6.501919
                17599       7.390371
                17864      10.136671
                18130      13.541893
                18396      13.781393
                18663      13.140681
                18928      12.037271
                19195      12.735506
                19460      16.052694
                19726      21.372453
                19992      27.427355
                20260      27.511936
                20525      21.792179
                20790      21.495848
                21056      26.565108
                21323      36.198404
                21589      43.982990
                21855      49.423444
                22121      74.043747
                22387      56.424411
                

In [11]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15791,2020-03-21,no,US,25825.0
16002,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33761.0
16267,2020-03-23,no,Germany,29056.0
16307,2020-03-23,no,US,43850.0
16535,2020-03-24,no,Germany,32986.0
16575,2020-03-24,no,US,54112.0
16800,2020-03-25,no,Germany,37323.0
16841,2020-03-25,no,US,66055.0


In [12]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg, 'confirmed').reset_index()

In [13]:
#pd_DR_result

In [14]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR','level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,466,NaN
2,Alberta,Canada,731,NaN
3,Alberta,Canada,965,NaN
4,Alberta,Canada,1233,NaN


In [15]:
pd_JH_data=pd_JH_data.reset_index()

In [16]:
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [17]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')

In [18]:
pd_result_larg

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN
...,...,...,...,...,...,...
46811,46811,2020-07-15,no,Barbados,104.0,206.666667
46812,46812,2020-07-15,no,Belarus,65443.0,396.810537
46813,46813,2020-07-15,no,Belgium,62872.0,761.050505
46814,46814,2020-07-15,no,Albania,3752.0,40.478821


In [19]:
#pd_result_larg[pd_result_larg['country']== 'Germany']
#to check if the ersults matches with the result generated on smaller datat set

## 4.3 Filtering the data with groupby apply

In [20]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [21]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [22]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')

In [23]:
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


## 4.4 Filtered doubling rate

In [24]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

In [25]:
pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
46811,no,Zimbabwe,45580,26.874159
46812,no,Zimbabwe,45848,29.500000
46813,no,Zimbabwe,46082,30.652352
46814,no,Zimbabwe,46348,35.119818
46815,no,Zimbabwe,46815,36.180887


In [26]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
46811,46811,2020-07-15,no,Barbados,104.0,206.666667,103.6,517.000000
46812,46812,2020-07-15,no,Belarus,65443.0,396.810537,65442.8,386.464772
46813,46813,2020-07-15,no,Belgium,62872.0,761.050505,62883.2,713.581818
46814,46814,2020-07-15,no,Albania,3752.0,40.478821,3758.0,37.543590
46815,46815,2020-07-15,no,Zimbabwe,1089.0,38.630303,1089.4,36.180887


Masks in python
<font color=green> When working with data arrays masks can be extremely useful. Masks are an array of boolean values for which a condition is met. </font>

In [27]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [28]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
45527,45527,2020-07-11,no,Germany,199709.0,563.128060,199628.2,652.030313
45795,45795,2020-07-12,no,Germany,199919.0,680.249858,199919.2,679.926658
46060,46060,2020-07-13,no,Germany,200180.0,848.985138,200230.8,663.544861
46326,46326,2020-07-14,no,Germany,200456.0,745.567970,200520.7,665.747520
46593,46593,2020-07-15,no,Germany,200890.0,564.813146,200810.6,691.689203


In [29]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)